<a href="https://colab.research.google.com/github/AdrianAlbrecht/Collab/blob/main/WS_zad_4_155376_Adrian_Albrecht.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importujemy potrzebne bilbioteki

In [1]:
import cv2
import numpy as np

Wczytujemy plik mp4 przycięty w czasie 7:31 - 8:31 z linku podanego w instrukcji zadania.

In [2]:
video_capture = cv2.VideoCapture('car_camera_clip.mp4')

# Konfigurujemy CV2 za pomocą zmiennych paramerrycznych

In [3]:
fps = video_capture.get(cv2.CAP_PROP_FPS)
frame_size = (int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT)))

# Sprawdźmy, czy plik da się otworzyć (może być uszkdozony, bądź w trakcie wczytywania pliku mogło się coś zepsuć)

In [4]:
if not video_capture.isOpened():
    print("Error: Could not open video file.")
    exit()

# Tworzymy potrzebne vmienne i obiekty do zaczęcia analizy

---
- Licznik klatek
- obiekt do zapisu video


In [5]:
frame_count = 0
output_video = cv2.VideoWriter('car_camera_with_red_lines.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, frame_size)

# Uruchamiamy główną pętlę algorytmu, w której klatka po klatce zaczniemy analizować klatkę krok po kroku według instrukcji algorytmu i zappisywanie nadmienione klatki

In [6]:
while True:
    ret, frame = video_capture.read()

    # Zakończ pętlę, jeżeli nie udało się wczytać klatki, co jest jednoznaczne z uszkodzeniem dalszej części pliku bądź końcu pliku
    if not ret:
        break

    # Dodaj do liczniki ramek analizowaną ramkę
    frame_count += 1

    # Konwersja klatki na szarość
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Rozmycie Gaussa na  klatce filmu
    gblur = cv2.GaussianBlur(gray, (15, 15), 0)

    # Wykrywanie krawędzi drogi na klatce algorytmem Canny'ego (low, high)
    gblur_canny = cv2.Canny(gblur, 20, 70)

    # Docinamy ramkę do fragmentu z drogą (normalnie 1920 x 1080)
    h, w, x, y = 290, 1650, 0, 700
    img1 = gblur_canny[y:y+h, x:x+w]
    img2 = np.zeros_like(gblur_canny)
    img2[y:y+h, x:x+w] = img1

    # Dodajemy czerwone linie oznaczające granice drogi na klatce
    dst = cv2.Canny(img2, 50, 200, None, 3)
    cdst = cv2.cvtColor(dst, cv2.COLOR_GRAY2BGR)
    cdstP = np.copy(cdst)

    lines = cv2.HoughLines(dst, 1, np.pi / 180, 150, None, 0, 0)
    if lines is not None:
        for i in range(0, len(lines)):
            rho = lines[i][0][0]
            theta = lines[i][0][1]
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a * rho
            y0 = b * rho
            pt1 = (int(x0 + 1000 * (-b)), int(y0 + 1000 * a))
            pt2 = (int(x0 - 1000 * (-b)), int(y0 - 1000 * a))

            cv2.line(cdst, pt1, pt2, (0, 0, 255), 3, cv2.LINE_AA)

    linesP = cv2.HoughLinesP(dst, 1, np.pi / 180, 50, None, 50, 10)
    if linesP is not None:
        for line in linesP:
            x1, y1, x2, y2 = line[0]
            cv2.line(frame, (x1, y1), (x2, y2), (0, 0, 255), 3, cv2.LINE_AA)

    # Podpisujemy klatkę i wyświetlamy numer ramki
    font = cv2.FONT_HERSHEY_TRIPLEX
    font2 = cv2.FONT_HERSHEY_SCRIPT_SIMPLEX
    font_scale = 0.8
    font_thickness = 2
    text_color = (255, 255, 0)
    text_color2 = (0, 0, 255)
    text_position_name = (frame_size[0] - 250, frame_size[1] - 20)
    cv2.putText(frame, "Adrian Albrecht", text_position_name, font, font_scale, text_color, font_thickness)
    text_position_frames_counter = (frame_size[0] - 1900, frame_size[1] - 1040)
    cv2.putText(frame, f'Ramka nr: {frame_count}', text_position_frames_counter,
                font2, font_scale, text_color2, font_thickness)

    # Wyświetlamy klatkę (opcjonalnie, Google Collab nie pozwala na tą funkcję)
    # cv2.imshow('Video', frame)

    # Zapisujemy obrobioną klatkę do pliku wyjściowego
    output_video.write(frame)

# Zwalniamy resztę zasobów, ponieważ, nie będą nam już potrzebne

In [7]:
video_capture.release()
output_video.release()
cv2.destroyAllWindows()